In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.ticker as ticker
from IPython.display import Audio
from sklearn import datasets, linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

sns.set()
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

In [2]:
#Features
features = pd.read_excel('Hot 100 Audio Features.xlsx')

In [3]:
#Billboard Top 100 By Week from 1958 to 2021
bb100 = pd.read_csv('Hot Stuff.csv')

In [4]:
#Convert 'WeekID' to DateTime Object & Check First & Last 2 entries by Week/Year
bb100['WeekID'] = pd.DatetimeIndex(bb100['WeekID'])
bb100.sort_values(by='WeekID').iloc[np.r_[0:2, -2:0]]

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
18553,http://www.billboard.com/charts/hot-100/1958-08-02,1958-08-02,63,High School Confidential,Jerry Lee Lewis And His Pumping Piano,High School ConfidentialJerry Lee Lewis And His Pumping Piano,1,NaN,63,1
103337,http://www.billboard.com/charts/hot-100/1958-08-02,1958-08-02,98,Little Serenade,The Ames Brothers,Little SerenadeThe Ames Brothers,1,NaN,98,1
300806,https://www.billboard.com/charts/hot-100/2021-05-29,2021-05-29,61,Almost Maybes,Jordan Davis,Almost MaybesJordan Davis,2,64.0,61,17
152154,https://www.billboard.com/charts/hot-100/2021-05-29,2021-05-29,78,White Teeth,YoungBoy Never Broke Again,White TeethYoungBoy Never Broke Again,1,NaN,78,1


In [5]:
#Merge BB100 with Features
bb100_features = pd.merge(bb100, features, on='SongID', how='left')
print(bb100_features.shape)
bb100_features.head(2)

(330461, 31)


,url,WeekID,Week Position,Song_x,Performer_x,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,Performer_y,Song_y,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,http://www.billboard.com/charts/hot-100/1965-07-17,1965-07-17,34,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,45.0,34,4,Patty Duke,Don't Just Stand There,['deep adult standards'],1YhNCQ3XOdTCZgubfX8PgB,NaN,163160.0,0.0,Lost Hits Of The 60's (All Original Artists & Versions),0.574,0.256,7.0,-15.044,1.0,0.0298,0.61,0.000077,0.1,0.568,82.331,3.0,21.0
1,http://www.billboard.com/charts/hot-100/1965-07-24,1965-07-24,22,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,34.0,22,5,Patty Duke,Don't Just Stand There,['deep adult standards'],1YhNCQ3XOdTCZgubfX8PgB,NaN,163160.0,0.0,Lost Hits Of The 60's (All Original Artists & Versions),0.574,0.256,7.0,-15.044,1.0,0.0298,0.61,0.000077,0.1,0.568,82.331,3.0,21.0


In [6]:
#Taking some important columns to introduce Spotify Metrics
subset_1 = bb100_features[['WeekID','Song_x', 'spotify_track_preview_url', 
                           'valence', 'spotify_track_popularity']]
print(subset_1.shape)
subset_1.head(2)

(330461, 5)


,WeekID,Song_x,spotify_track_preview_url,valence,spotify_track_popularity
0,1965-07-17,Don't Just Stand There,NaN,0.568,21.0
1,1965-07-24,Don't Just Stand There,NaN,0.568,21.0


In [7]:
#Remove all Nan's from subset_1
subset_1 = subset_1.dropna()
print(subset_1.shape)
subset_1.head(2)

(169763, 5)


,WeekID,Song_x,spotify_track_preview_url,valence,spotify_track_popularity
14,1971-04-24,Don't Knock My Love - Pt. 1,https://p.scdn.co/mp3-preview/5d3332b4ae616cd2157f948f7329b20470714a95?cid=b8d3901151d34489a160e3cf0ab1fa94,0.961,26.0
15,1971-05-01,Don't Knock My Love - Pt. 1,https://p.scdn.co/mp3-preview/5d3332b4ae616cd2157f948f7329b20470714a95?cid=b8d3901151d34489a160e3cf0ab1fa94,0.961,26.0


In [8]:
#Remove all duplicate songs from subset_1
subset_1 = subset_1.drop_duplicates(subset='Song_x')
print(subset_1.shape)
subset_1.head(2)

(12689, 5)


,WeekID,Song_x,spotify_track_preview_url,valence,spotify_track_popularity
14,1971-04-24,Don't Knock My Love - Pt. 1,https://p.scdn.co/mp3-preview/5d3332b4ae616cd2157f948f7329b20470714a95?cid=b8d3901151d34489a160e3cf0ab1fa94,0.961,26.0
39,1989-09-30,Don't Know Much,https://p.scdn.co/mp3-preview/49fff23849dc6ec86909dd299908100857831882?cid=b8d3901151d34489a160e3cf0ab1fa94,0.267,60.0


In [9]:
#Sort by Valence Score
Val = subset_1.sort_values(by='valence', ascending=False)
Val.head(3)

,WeekID,Song_x,spotify_track_preview_url,valence,spotify_track_popularity
54516,1961-06-17,I'm Gonna Knock On Your Door,https://p.scdn.co/mp3-preview/8adff0cd5d7a7fec39c74112e51aff37c1aea442?cid=b8d3901151d34489a160e3cf0ab1fa94,0.991,31.0
103279,1976-07-31,Hideaway,https://p.scdn.co/mp3-preview/d2eb277a2edc1deb8bd930da3896d553fec090ab?cid=b8d3901151d34489a160e3cf0ab1fa94,0.988,9.0
2882,1968-01-27,Simon Says,https://p.scdn.co/mp3-preview/432123ffa77a81724a930cca8c3072842d19c922?cid=b8d3901151d34489a160e3cf0ab1fa94,0.985,30.0


In [10]:
Val = Val.loc[(Val['valence']>0.85) &
              (Val['spotify_track_popularity'] > 65)]
Val.shape

(196, 5)

In [11]:
Val = Val.sort_values(by='WeekID').head(10)
Val

,WeekID,Song_x,spotify_track_preview_url,valence,spotify_track_popularity
54009,1962-08-11,Green Onions,https://p.scdn.co/mp3-preview/6afcf8c2efdb3c33264e8aa5836927df8b65b78c?cid=b8d3901151d34489a160e3cf0ab1fa94,0.912,66.0
51539,1963-11-09,Louie Louie,https://p.scdn.co/mp3-preview/8bae8308d070c28c91837da1d467a0d03acc3cc4?cid=b8d3901151d34489a160e3cf0ab1fa94,0.948,68.0
201420,1964-09-26,You Really Got Me,https://p.scdn.co/mp3-preview/acc77ee586d98989a59d5d0ec308b01d06ad995c?cid=b8d3901151d34489a160e3cf0ab1fa94,0.963,73.0
8102,1966-12-10,I'm A Believer,https://p.scdn.co/mp3-preview/4e143a9e5486ecc18bfdfac1b4f5d9d2d730a328?cid=b8d3901151d34489a160e3cf0ab1fa94,0.962,73.0
6150,1967-04-29,Respect,https://p.scdn.co/mp3-preview/b0d0b4ec8963779239654542a0b33d25caf38156?cid=b8d3901151d34489a160e3cf0ab1fa94,0.965,73.0
1543,1967-08-12,(Your Love Keeps Lifting Me) Higher And Higher,https://p.scdn.co/mp3-preview/715dfaeb3221769b58a3712d492a68f979535883?cid=b8d3901151d34489a160e3cf0ab1fa94,0.938,70.0
3795,1967-09-02,Brown Eyed Girl,https://p.scdn.co/mp3-preview/cb2b0096b08796c0c1cfeb31b9a9802ce0a3b44a?cid=b8d3901151d34489a160e3cf0ab1fa94,0.907,80.0
83644,1968-05-18,Think,https://p.scdn.co/mp3-preview/98251aa932f7f5cb0fea00c8abda5d0f33630139?cid=b8d3901151d34489a160e3cf0ab1fa94,0.926,68.0
103213,1968-05-25,Folsom Prison Blues,https://p.scdn.co/mp3-preview/cfc6157d9618d8598592c05c6cb8fb1e5b7190e3?cid=b8d3901151d34489a160e3cf0ab1fa94,0.889,70.0
9711,1969-01-04,Build Me Up Buttercup,https://p.scdn.co/mp3-preview/a1286ea4146157ad1559eaefe6e8a0869e3963b8?cid=b8d3901151d34489a160e3cf0ab1fa94,0.852,73.0


In [12]:
#Get 'I'm A Believer' preview URL:
Bel = Val['spotify_track_preview_url'].iloc[3]
Bel

'https://p.scdn.co/mp3-preview/4e143a9e5486ecc18bfdfac1b4f5d9d2d730a328?cid=b8d3901151d34489a160e3cf0ab1fa94'

In [13]:
#Download URL and save as MP3 file: 
import urllib.request
url = Bel
filename = 'Val_Score_0.962.mp3'
urllib.request.urlretrieve(url, filename)

('Val_Score_0.962.mp3', <http.client.HTTPMessage at 0x7f988093d220>)

In [14]:
#Play MP3 file: 
from IPython.display import Audio
Audio('Val_Score_0.962.mp3')

In [15]:
#Sort by Valence Score
Val2 = subset_1.sort_values(by='valence', ascending=False)
Val2.head(3)

,WeekID,Song_x,spotify_track_preview_url,valence,spotify_track_popularity
54516,1961-06-17,I'm Gonna Knock On Your Door,https://p.scdn.co/mp3-preview/8adff0cd5d7a7fec39c74112e51aff37c1aea442?cid=b8d3901151d34489a160e3cf0ab1fa94,0.991,31.0
103279,1976-07-31,Hideaway,https://p.scdn.co/mp3-preview/d2eb277a2edc1deb8bd930da3896d553fec090ab?cid=b8d3901151d34489a160e3cf0ab1fa94,0.988,9.0
2882,1968-01-27,Simon Says,https://p.scdn.co/mp3-preview/432123ffa77a81724a930cca8c3072842d19c922?cid=b8d3901151d34489a160e3cf0ab1fa94,0.985,30.0


In [16]:
Val2 = Val2.loc[(Val2['valence']<0.30) &
              (Val2['spotify_track_popularity'] > 85)]
Val2.shape

(12, 5)

In [17]:
Val2

,WeekID,Song_x,spotify_track_preview_url,valence,spotify_track_popularity
7477,2019-12-28,Futsal Shuffle 2020,https://p.scdn.co/mp3-preview/ab9ea4c65ee7c5ef65a55c1667ca522114da4c5c?cid=b8d3901151d34489a160e3cf0ab1fa94,0.2960,91.0
33114,2019-03-30,Slow Dancing In The Dark,https://p.scdn.co/mp3-preview/483355f39bb264b9828633561ab14a7a48e75270?cid=b8d3901151d34489a160e3cf0ab1fa94,0.2840,86.0
7661,2019-10-26,Lights Up,https://p.scdn.co/mp3-preview/2a91b13bd532e2752e0c48d23a459b873d9bf530?cid=b8d3901151d34489a160e3cf0ab1fa94,0.2700,87.0
1386,2019-06-22,Callaita,https://p.scdn.co/mp3-preview/23c0b25cf824c1bfd7a814fe7cd1264c31a5a9dc?cid=b8d3901151d34489a160e3cf0ab1fa94,0.2440,91.0
2463,2010-12-18,Christmas Lights,https://p.scdn.co/mp3-preview/83bc99b80b9bd0115cbd1ff6447152f60f8c4386?cid=b8d3901151d34489a160e3cf0ab1fa94,0.2370,86.0
4632,2019-12-07,Suicidal,https://p.scdn.co/mp3-preview/b7b850b2189ee14f3bec0c5e9313688d4b3d157e?cid=b8d3901151d34489a160e3cf0ab1fa94,0.2350,88.0
41647,2021-03-06,The Business,https://p.scdn.co/mp3-preview/676cb431611544326bb3e81c72d8dc5b27e0c9ad?cid=b8d3901151d34489a160e3cf0ab1fa94,0.2350,93.0
4136,2021-03-13,Streets,https://p.scdn.co/mp3-preview/72699a9f3ab9479740438e33cbbd4af05777329c?cid=b8d3901151d34489a160e3cf0ab1fa94,0.1900,91.0
18669,2020-05-30,Martin & Gina,https://p.scdn.co/mp3-preview/c3a29031fae3d7ec39e63efa734da8595be6a916?cid=b8d3901151d34489a160e3cf0ab1fa94,0.1160,87.0
296314,2020-09-05,You Broke Me First.,https://p.scdn.co/mp3-preview/b38949dd08a15c916b0ae764f74ac07b05af332b?cid=b8d3901151d34489a160e3cf0ab1fa94,0.0823,88.0


In [18]:
#Get 'Falling' preview URL:
Fall = Val2['spotify_track_preview_url'].iloc[-1]
Fall

'https://p.scdn.co/mp3-preview/755e88e49f038885df13efc6689381a2068e972d?cid=b8d3901151d34489a160e3cf0ab1fa94'

In [19]:
#Download URL and save as MP3 file: 
import urllib.request
url = Fall
filename = 'Val_Score_0.0592.mp3'
urllib.request.urlretrieve(url, filename)

('Val_Score_0.0592.mp3', <http.client.HTTPMessage at 0x7f988093da60>)

In [20]:
#Play MP3 file: 
from IPython.display import Audio
Audio('Val_Score_0.0592.mp3')